**Process pedestrian JSON file**

In [1]:
%matplotlib inline  
import os
import numpy as np
from PIL import Image
import cv2
from matplotlib.pyplot import figure, imshow, savefig
import matplotlib.pyplot as plt
import collections
import os
import json
import torch

In [4]:
fold_dict_filename = 'pedestrian_dataset_folds/fold_dict.json'
fold_dict = json.load(open(fold_dict_filename, 'r'))
num_frames = 30
frames_to_process = set()
for json_filename in fold_dict:
    json_path = os.path.join(fold_dict[json_filename], json_filename)
    ped_json = json.load(open(json_path, 'r'))
    video_name = ped_json['video']
    first_frame = ped_json['frame_data'][0]
    start = first_frame['frame_index']
    for idx in range(start, start + num_frames):
        frames_to_process.add(video_name + '-' + str(idx))

In [16]:
frames_dict = {}
for frame in frames_to_process:
    video, idx = frame.split('-')
    if video not in frames_dict:
        frames_dict[video] = []
    frames_dict[video].append(int(idx))
for video in frames_dict:
    frames_dict[video] = sorted(frames_dict[video])

In [37]:
names_dict = {}
for video in frames_dict:
    if video not in names_dict:
        names_dict[video] = []
    for idx in frames_dict[video]:  
        names_dict[video].append(video + '-' + str(idx))

In [30]:
video_id = 'video_0266'
video_name = os.path.join('clips', video_id + '.mp4')
images = get_video_frames(video_name, frames_dict[video_id])

In [ ]:
for image in enumerate(i, images):
    image_name = names_dict[video_id][i]
    test(image, model, 19, image_name)

In [44]:
names_dict['video_0266']

['video_0266-17',
 'video_0266-18',
 'video_0266-19',
 'video_0266-20',
 'video_0266-21',
 'video_0266-22',
 'video_0266-23',
 'video_0266-24',
 'video_0266-25',
 'video_0266-26',
 'video_0266-27',
 'video_0266-28',
 'video_0266-29',
 'video_0266-30',
 'video_0266-31',
 'video_0266-32',
 'video_0266-33',
 'video_0266-34',
 'video_0266-35',
 'video_0266-36',
 'video_0266-37',
 'video_0266-38',
 'video_0266-39',
 'video_0266-40',
 'video_0266-41',
 'video_0266-42',
 'video_0266-43',
 'video_0266-44',
 'video_0266-45',
 'video_0266-46',
 'video_0266-47',
 'video_0266-48',
 'video_0266-49',
 'video_0266-50',
 'video_0266-51',
 'video_0266-52',
 'video_0266-53',
 'video_0266-54',
 'video_0266-55',
 'video_0266-56',
 'video_0266-57',
 'video_0266-58',
 'video_0266-59',
 'video_0266-60',
 'video_0266-61',
 'video_0266-62',
 'video_0266-63',
 'video_0266-64',
 'video_0266-65',
 'video_0266-66',
 'video_0266-67',
 'video_0266-68',
 'video_0266-73',
 'video_0266-74',
 'video_0266-75',
 'video_02